In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os

In [21]:
### El objetivo es obtener una planilla con los promedios mensuales de niveles de 20 pozos para una calibracion de un modelo con stress period mensuales.
### Se tienen datos piezometricos de los 20 pozos con registros diversos (diarios, horarios, etc) entre 2010 y 2015
### Tambien se necesita graficar la evolucion temporal observada

In [24]:
input_file = "piezometric_levels.txt"
output_file = "piezometric_monthly_avgs.txt"
plot_dir = "piezometer_plots"

In [47]:
def process():
    #leer el archivo
    df = pd.read_csv(input_file, sep=",")

    #agrupar por pozo y fecha
    df["date"] = pd.to_datetime(df["date"], errors="coerce", infer_datetime_format=True)

    #eliminar filas con valores nulos en date o level_m
    df = df.dropna(subset=["date", "level_m"])

    #calcular el promedio mensual por pozo
    monthly = (
        df.set_index("date") # usa date como indice temporal
        .groupby("well") # agrupa por pozo
        .resample("MS") # reescala por mes (month start)
        .mean(numeric_only=True) #promedio mensual
        .reset_index() # convierte el indice como una nueva columna
        .pivot(index="date", columns="well", values="level_m") #cambia la forma de la matriz
        .sort_index() # ordena el indice temporal
    )
    #exportar nueva matriz como txt
    monthly.to_csv(output_file, index=True)

    # crear directorio para guardar los graficos
    os.makedirs(plot_dir, exist_ok=True)

    #graficar la evolucion temporal de cada pozo como un plot de lineas
    for well, g in df.groupby("well"):
        g = g.sort_values("date") # ordena por fecha
        plt.figure(figsize=(10,4))
        plt.plot(g["date"], g["level_m"], linestyle="-")
        plt.xlabel("Fecha")
        plt.ylabel("Nivel observado (m)")
        plt.title(f"Piezometro {well}: Nivel observado")
        plt.grid(True)

        #fijar el rango de eje x
        plt.xlim(pd.to_datetime("2010-01-01"), pd.to_datetime("2026-01-01"))
        ax=plt.gca()
        ax.xaxis.set_major_locator(mdates.YearLocator(2, month=1, day=1))
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))

        #fijar el rango de eje y
        plt.ylim(300,600)
        
        plt.tight_layout()
        #guardar la figura en el directorio de plots
        plt.savefig(os.path.join(plot_dir, f"{well}.png"))
        plt.close()

    return df, monthly


In [48]:
process()

C:\Users\jvc96\AppData\Local\Temp\ipykernel_34516\3193790580.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["date"] = pd.to_datetime(df["date"], errors="coerce", infer_datetime_format=True)


(        well       date     level_m
 0       PZ01 2010-01-24  420.618874
 1       PZ01 2010-01-25  425.001579
 2       PZ01 2010-01-26  416.221661
 3       PZ01 2010-01-27  416.227741
 4       PZ01 2010-01-28  425.300488
 ...      ...        ...         ...
 570093  PZ20 2025-07-08  451.839170
 570094  PZ20 2025-07-09  448.323271
 570095  PZ20 2025-07-10  453.032426
 570096  PZ20 2025-07-11  452.987343
 570097  PZ20 2025-07-12  454.470975
 
 [33258 rows x 3 columns],
 well              PZ01  PZ02  PZ03  PZ05  PZ08  PZ10  PZ11  PZ13  PZ14  PZ15  \
 date                                                                           
 2010-01-01  419.977776   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
 2010-02-01  415.349616   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
 2010-03-01  417.497542   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
 2010-04-01  417.641739   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
 2010-05-01  418.269827   NaN   NaN   NaN